# Assignment 1: Wrangling and EDA
### Foundations of Machine Learning

In [ ]:
! git clone https://github.com/ds4e/scratchpad
%run ./scratchpad/get_data.py

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
files.upload()

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `airbnb_NYC.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

The Price variable was stored as an object because prices over 999 contain commas (e.g., “1,112”). I removed commas and converted the column to numeric, which allowed all values to be retained. After cleaning, there were 0 missing values.

2. Categorical variable: For the Minnesota police use of for data, `mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? For the remaining missing values, replace the `np.nan/None` values with the label `Missing`.

The subject_injury variable was cleaned by keeping “Yes” and “No” values and labeling missing entries as “Missing.” About 76% of the values were missing. Cross-tabulation with force_type shows that missing injury data are more common for certain force types, such as Bodily Force and Chemical Irritant, suggesting that missingness is related to the type of force used rather than being random.

3. Dummy variable: For `metabric.csv`, convert the `Overall Survival Status` variable into a dummy/binary variable, taking the value 0 if the patient is deceased and 1 if they are living.

The Overall Survival Status variable was converted into a binary variable by assigning a value of 0 to deceased patients and 1 to living patients. This required matching the original string labels in the dataset (1:DECEASED and 0:LIVING) to their corresponding numeric values. Creating this dummy variable makes the survival outcome easier to use in analysis and modeling.

4. Missing values: For `airbnb_NYC.csv`, determine how many missing values of `Review Scores Rating` there are. Create a new variable, in which you impute the median score for non-missing observations to the missing ones. Why might this bias or otherwise negatively impact your results?

There are 8,323 missing values in the Review Scores Rating variable. I created a new variable by imputing the median rating of 94 for the missing values. This approach can bias results by reducing variability and hiding meaningful differences between listings, especially if missing ratings are not random.

In [ ]:
airbnb = pd.read_csv("airbnb_NYC.csv")

In [ ]:
airbnb = pd.read_csv("airbnb_NYC.csv", encoding="latin1")

In [ ]:
airbnb.columns

In [ ]:
airbnb["Price"].head(10)


In [ ]:
airbnb.head()

In [ ]:
airbnb["Price"].head(10)
airbnb["Price"].dtype

In [ ]:
airbnb["Price_clean"] = (
    airbnb["Price"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .astype(float)
)


In [ ]:
airbnb["Price_clean"].dtype
airbnb["Price_clean"].isna().sum()

In [ ]:
mn = pd.read_csv("mn_police_use_of_force.csv", encoding="latin1")

In [ ]:
from google.colab import files
files.upload()

In [ ]:
mn = pd.read_csv("mn_police_use_of_force.csv", encoding="latin1")

In [ ]:
mn.head()

In [ ]:
mn["subject_injury"].value_counts(dropna=False)

In [ ]:
mn["subject_injury_clean"] = mn["subject_injury"].replace({
    "Yes": "Yes",
    "No": "No",
    np.nan: "Missing"
})

In [ ]:
mn["subject_injury_clean"].value_counts()

In [ ]:
missing_prop = (mn["subject_injury_clean"] == "Missing").mean()
missing_prop

In [ ]:
pd.crosstab(mn["force_type"], mn["subject_injury_clean"])

In [ ]:
from google.colab import files
files.upload()

In [ ]:
meta = pd.read_csv("metabric.csv", encoding="latin1")

In [ ]:
meta["Overall Survival Status"].value_counts()

In [ ]:
meta["survival_binary"] = meta["Overall Survival Status"].map({
    "Deceased": 0,
    "Living": 1
})

In [ ]:
meta[["Overall Survival Status", "survival_binary"]].head()

In [ ]:
meta["survival_binary"].value_counts()

In [ ]:
meta["survival_binary"] = meta["Overall Survival Status"].map({
    "1:DECEASED": 0,
    "0:LIVING": 1
})

In [ ]:
meta[["Overall Survival Status", "survival_binary"]].head()

In [ ]:
meta["survival_binary"].value_counts()

In [ ]:
airbnb["Review Scores Rating"].isna().sum()

In [ ]:
median_rating = airbnb["Review Scores Rating"].median()
median_rating

In [ ]:
airbnb["Review_Scores_Imputed"] = airbnb["Review Scores Rating"].fillna(median_rating)

In [ ]:
airbnb["Review_Scores_Imputed"].isna().sum()

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work. What does work?

The shark attack dataset is an Excel file rather than a CSV, so read_csv does not work. Instead, I used pd.read_excel() to load the data successfully into pandas and verified it by viewing the first few rows and the dataset shape.

2. Drop any columns that do not contain data.

After inspecting missing values by column, I found that several columns contained no meaningful data. I dropped columns that were entirely missing using dropna(axis=1, how="all"), which reduced the dataset to only columns with at least some observed values.

3. What is an observation? Carefully justify your answer, and explain how it affects your choices in cleaning and analyzing the data.

Each observation represents a single shark attack incident, with one row corresponding to one event. This affects the analysis by guiding cleaning decisions to preserve one row per attack while standardizing variables like year, age, attack type, and fatality status for consistent comparison across incidents.

4. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?

The Year variable was cleaned by converting it to numeric and removing invalid values. The cleaned year values range from 1940 to recent years. After filtering the data to include only attacks since 1940, the number of recorded shark attacks generally increases over time, with noticeable year-to-year variation. The apparent drop in the most recent years is likely due to incomplete reporting rather than an actual decrease in attacks.

5. Clean the Age variable and make a histogram of the ages of the victims.
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?

74%

7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Thoughts?

Unprovoked shark attacks are more likely to be fatal than provoked attacks. The cross-tabulation shows that about 24% of unprovoked attacks result in fatalities, compared to only about 3% of provoked attacks. This suggests that unprovoked encounters tend to be more severe, while provoked incidents are less likely to lead to fatal outcomes.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
sharks = pd.read_excel("GSAF5.xls")

In [ ]:
sharks.head()
sharks.shape

In [ ]:
sharks.isna().sum().sort_values(ascending=False).head(20)

In [ ]:
sharks.isna().sum().sort_values(ascending=False).head(20)

In [ ]:
sharks = sharks.dropna(axis=1, how="all")

In [ ]:
sharks.shape

In [ ]:
sharks["Year"].head(10)
sharks["Year"].dtype


In [ ]:
sharks["Year_clean"] = pd.to_numeric(sharks["Year"], errors="coerce")


In [ ]:
sharks["Year_clean"].describe()

In [ ]:
sharks_since_1940 = sharks[sharks["Year_clean"] >= 1940]

In [ ]:
sharks_since_1940.shape

In [ ]:
sharks_since_1940["Year_clean"].value_counts().sort_index().plot()

In [ ]:
sharks["Age"].head(10)
sharks["Age"].dtype

In [ ]:
sharks["Age_clean"] = pd.to_numeric(sharks["Age"], errors="coerce")

In [ ]:
sharks["Age_clean"].describe()

In [ ]:
sharks["Age_clean"].dropna().plot(kind="hist", bins=20)

In [ ]:
sharks["Type"].value_counts(dropna=False)


In [ ]:
sharks["Type_clean"] = sharks["Type"].replace({
    "Provoked": "Provoked",
    "Unprovoked": "Unprovoked"
})


In [ ]:
sharks["Type_clean"] = sharks["Type_clean"].fillna("Unknown")


In [ ]:
sharks["Type_clean"].value_counts()


In [ ]:
sharks["Type_clean"].value_counts()


In [ ]:
sharks["Type_clean"] = sharks["Type"].str.strip().str.capitalize()


In [ ]:
sharks["Type_clean"] = sharks["Type_clean"].where(
    sharks["Type_clean"].isin(["Provoked", "Unprovoked"]),
    "Unknown"
)

In [ ]:
sharks["Type_clean"].value_counts()


In [ ]:
sharks["Fatal Y/N"].value_counts(dropna=False)


In [ ]:
sharks["Fatal_clean"] = sharks["Fatal Y/N"].str.strip().str.upper()


In [ ]:
sharks["Fatal_clean"] = sharks["Fatal_clean"].where(
    sharks["Fatal_clean"].isin(["Y", "N"]),
    "Unknown"
)


In [ ]:
sharks["Fatal_clean"].value_counts()


In [ ]:
pd.crosstab(
    sharks["Type_clean"],
    sharks["Fatal_clean"],
    normalize="index"
)


**Q3.** Open the "tidy_data.pdf" document available in `https://github.com/ds4e/wrangling`, which is a paper called *Tidy Data* by Hadley Wickham.

  1. Read the abstract. What is this paper about?

  The paper argues that many difficulties in data analysis come from poor data structure rather than from modeling or statistical methods. Wickham introduces the concept of tidy data and proposes a simple, consistent way to organize datasets so they are easier to manipulate, visualize, and analyze.

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?

  The tidy data standard is meant to reduce friction in data analysis by providing a common structure for datasets. By organizing data so that variables, observations, and values are clearly defined and consistently placed, the standard makes data cleaning more systematic and allows analysts to spend less time reshaping data and more time analyzing it.

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

  The statement “Like families, tidy datasets are all alike but every messy dataset is messy in its own way” means that tidy datasets follow a shared structure, while messy datasets can be disorganized in many different and unpredictable forms. The follow-up sentence emphasizes that while it is often easy to roughly identify what the variables and observations are in a dataset, it is much harder to define them precisely and consistently. This lack of precision is what leads to messy data structures that complicate analysis.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?

  Values as the actual measurements or entries in the dataset. Variables as the attributes being measured (what is recorded). Observations as a single unit across all variables (what the measurements describe).

  5. How is "Tidy Data" defined in section 2.3?
  
  Wickham defines tidy data using three rules: each variable must have its own column, each observation must have its own row, and each type of observational unit must form its own table. When these rules are followed, data become easier to transform, visualize, and model.

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?

  Wickham identifies five common problems with messy datasets: column headers that contain values instead of variable names, multiple variables stored in a single column, variables stored in both rows and columns, multiple types of observational units combined in one table, and a single observational unit split across multiple tables. Table 4 is messy because it violates these tidy data principles, particularly by embedding values in column headers and mixing multiple variables within the same structure, which makes it difficult to clearly identify variables and observations and complicates analysis.

  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

  Table 11 is messy because it spreads observations across multiple columns and embeds variable names as values, making the structure difficult to analyze. Table 12 is tidy and “molten” because the data have been reshaped into a long format where each variable has its own column and each observation has its own row. This molten structure follows tidy data principles and supports more flexible analysis.

**Q4.** This question looks at financial transfers from international actors to American universities. In particular, from which countries and giftors are the gifts coming from, and to which institutions are they going?

For this question, `.groupby([vars]).count()` and `.groupby([vars]).sum()` will be especially useful to tally the number of occurrences and sum the values of those occurrences.

1. Load the `ForeignGifts_edu.csv` dataset.

The ForeignGifts_edu.csv dataset was loaded into a pandas DataFrame called gifts. The dataset contains information on foreign financial transfers to U.S. higher education institutions, including the institution name, gift amount, gift type, country of the giftor, and the giftor name. Each row represents a reported foreign gift or contract.

2. For `Foreign Gift Amount`, create a histogram and describe the variable. Describe your findings.

The variable Foreign Gift Amount is highly right-skewed. Most foreign gifts are relatively small in dollar value, while a small number of gifts are extremely large, reaching into the hundreds of millions of dollars. This indicates that although many foreign gifts occur, the total amount of funding is driven by a small number of very large transfers rather than typical-sized gifts.

3. For `Gift Type`, create a histogram or value counts table. What proportion of the gifts are contracts, real estate, and monetary gifts?

When examining Gift Type, the majority of foreign financial transfers fall into monetary gifts and contracts, with real estate gifts making up a much smaller proportion. Contracts account for a substantial share of the total value, reflecting formal agreements between foreign entities and U.S. institutions, while purely monetary gifts are more frequent but vary widely in size.

4. What are the top 15 countries in terms of the number of gifts? What are the top 15 countries in terms of the amount given?

In terms of the number of gifts, England, China, and Canada are the most frequent sources of foreign gifts to U.S. universities. However, when ranked by the total dollar amount, the ordering changes significantly. Qatar contributes the largest total amount despite having fewer gifts, followed by England, China, and Saudi Arabia. This shows that some countries provide fewer but much larger gifts, while others contribute more frequently but in smaller amounts.

5. What are the top 15 institutions in terms of the total amount of money they receive? Make a histogram of the total amount received by all institutions.

Foreign gift funding is heavily concentrated among a small number of elite institutions. Carnegie Mellon University and Cornell University receive the largest total amounts of foreign funding, followed by Harvard University and MIT. The histogram of total foreign gifts by institution is extremely right-skewed, showing that most institutions receive relatively small amounts, while a small number receive exceptionally large sums.

6. Which giftors provide the most money, in total?

The largest foreign giftors are dominated by a small group of organizations. The Qatar Foundation is the single largest contributor by total amount, followed by other Qatar-affiliated entities such as the Qatar National Research Fund and Qatar Foundation for Education. Several Saudi-affiliated organizations and government entities also appear among the top contributors, along with a notable amount of funding coming from anonymous donors. Overall, foreign funding is concentrated among a limited set of large, often state-linked or foundation-based giftors.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
files.upload()

In [ ]:
gifts = pd.read_csv("ForeignGifts_edu.csv")
gifts.head()

In [ ]:
gifts["Foreign Gift Amount"].describe()

In [ ]:
gifts["Foreign Gift Amount"].plot(kind="hist", bins=50)
plt.xlabel("Foreign Gift Amount")
plt.title("Distribution of Foreign Gift Amounts")
plt.show()

In [ ]:
gift_counts = gifts["Gift Type"].value_counts()
gift_counts

In [ ]:
gift_counts / gift_counts.sum()

In [ ]:
top_countries_count = (
    gifts.groupby("Country")
    .size()
    .sort_values(ascending=False)
    .head(15)
)

top_countries_count

In [ ]:
gifts.columns

In [ ]:
top_countries_count = (
    gifts.groupby("Country of Gift Origin")
    .size()
    .sort_values(ascending=False)
    .head(15)
)

top_countries_count

In [ ]:
top_countries_count = (
    gifts.groupby("Country of Giftor")
    .size()
    .sort_values(ascending=False)
    .head(15)
)

top_countries_count

In [ ]:
top_countries_amount = (
    gifts.groupby("Country of Giftor")["Foreign Gift Amount"]
    .sum()
    .sort_values(ascending=False)
    .head(15)
)

top_countries_amount

In [ ]:
top_institutions = (
    gifts.groupby("Institution Name")["Foreign Gift Amount"]
    .sum()
    .sort_values(ascending=False)
    .head(15)
)

top_institutions

In [ ]:
institution_totals = (
    gifts.groupby("Institution Name")["Foreign Gift Amount"]
    .sum()
)

In [ ]:
institution_totals.plot(kind="hist", bins=50)
plt.xlabel("Total Foreign Gift Amount Received")
plt.title("Distribution of Total Foreign Gifts by Institution")
plt.show()


In [ ]:
top_giftors = (
    gifts.groupby("Giftor Name")["Foreign Gift Amount"]
    .sum()
    .sort_values(ascending=False)
    .head(15)
)

top_giftors

In [ ]:
college.shape
college.head()

**Q5.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `college_completion.csv` dataset from the US Department of Education. The "relevant" variables for this question are:
  - `level` - Level of institution (4-year, 2-year)
  - `aid_value` - The average amount of student aid going to undergraduate recipients
  - `control` - Public, Private not-for-profit, Private for-profit
  - `grad_100_value` - percentage of first-time, full-time, degree-seeking undergraduates who complete a degree or certificate program within 100 percent of expected time (bachelor's-seeking group at 4-year institutions)

1. Load the `college_completion.csv` data with Pandas.
2. How many observations and variables are in the data? Use `.head()` to examine the first few rows of data.

The dataset contains 3,798 observations and 63 variables. Each observation corresponds to a U.S. postsecondary institution, and the variables capture institutional characteristics including location, level, control, student aid, and graduation outcomes. Inspecting the first few rows confirms that the data include a mix of public and private institutions across different states and institutional types.

3. Cross tabulate `control` and `level`. Describe the patterns you see in words.

The cross-tabulation shows clear differences in institutional structure across control types. Private not-for-profit institutions are overwhelmingly 4-year schools, with about 94.6% offering 4-year programs and only 5.4% being 2-year institutions. Public institutions are more evenly split, though they lean toward 2-year colleges, with roughly 59.4% being 2-year and 40.6% being 4-year, reflecting the prevalence of community colleges in the public sector. Private for-profit institutions are the most balanced across levels, with about 53.1% being 4-year and 46.9% being 2-year institutions. Overall, institutional control is strongly associated with institutional level, with not-for-profit schools concentrated in the 4-year sector and public schools playing a major role in 2-year education.

4. For `grad_100_value`, create a kernel density plot and describe table. Now condition on `control`, and produce a kernel density plot and describe tables for each type of institutional control. Which type of institution appear to have the most favorable graduation rates?

They have the highest mean and median graduation rates, and their distribution is shifted to the right compared to public and private for-profit schools. Public institutions have the lowest graduation rates overall, while private for-profit institutions fall in between but show more variability.

5. Make a scatterplot of `grad_100_value` by `aid_value`, and compute the covariance and correlation between the two variables. Describe what you see. Now make the same plot and statistics, but conditioning on `control`. Describe what you see. For which kinds of institutions does aid seem to vary positively with graduation rates?

Looking at the scatterplot, there’s an overall upward trend between average student aid and graduation rates, but it’s pretty noisy. When breaking it out by institutional control, the pattern becomes clearer. Private not-for-profit schools show the strongest positive relationship between aid and graduation rates, where higher aid generally lines up with higher completion rates. Public institutions also show a positive relationship, though it’s weaker and more spread out. Private for-profit schools show only a very weak relationship, with graduation rates staying low even as aid increases. Overall, aid seems to vary most positively with graduation rates at private not-for-profit institutions, followed by public schools, and least for private for-profit schools.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

college = pd.read_csv("college_completion.csv")


In [ ]:
college.shape

In [ ]:
pd.crosstab(college["control"], college["level"])

In [ ]:
pd.crosstab(college["control"], college["level"], normalize="index")

In [ ]:
college["grad_100_value"].describe()

In [ ]:
college["grad_100_value"].plot(kind="kde")
plt.xlabel("Graduation rate (100% time)")
plt.title("Kernel Density of Graduation Rates")
plt.show()

In [ ]:
college.groupby("control")["grad_100_value"].describe()

In [ ]:
college.groupby("control")["grad_100_value"].plot(kind="kde", legend=True)
plt.xlabel("Graduation rate (100% time)")
plt.title("Graduation Rates by Institutional Control")
plt.show()

In [ ]:
plt.scatter(college["aid_value"], college["grad_100_value"], alpha=0.4)
plt.xlabel("Average Student Aid (aid_value)")
plt.ylabel("Graduation Rate (100% time)")
plt.title("Graduation Rate vs Student Aid")
plt.show()

In [ ]:
cov = college[["aid_value", "grad_100_value"]].cov()
corr = college[["aid_value", "grad_100_value"]].corr()

cov, corr

In [ ]:
for c in college["control"].unique():
    subset = college[college["control"] == c]
    plt.scatter(subset["aid_value"], subset["grad_100_value"], alpha=0.5, label=c)

plt.xlabel("Average Student Aid (aid_value)")
plt.ylabel("Graduation Rate (100% time)")
plt.title("Graduation Rate vs Aid by Institutional Control")
plt.legend()
plt.show()

In [ ]:
college.groupby("control")[["aid_value", "grad_100_value"]].corr()

In [ ]:
college.groupby("control").apply(
    lambda df: df["aid_value"].corr(df["grad_100_value"])
)

In [ ]:
import numpy as np


**Q6.** In class, we talked about how to compute the sample mean of a variable $X$,
$$
m(X) = \dfrac{1}{N} \sum_{i=1}^N x_i
$$
and sample covariance of two variables $X$ and $Y$,
$$
\text{cov}(X,Y) = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))(y_i - m(Y))).
$$
Recall, the sample variance of $X$ is
$$
s^2 = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))^2.
$$
It can be very helpful to understand some basic properties of these statistics. If you want to write your calculations on a piece of paper, take a photo, and upload that to your GitHub repo, that's probably easiest.

We're going to look at **linear transformations** of $X$, $Y = a + bX$. So we take each value of $X$, $x_i$, and transform it as $y_i = a + b x_i$.

1. Show that $m(a + bX) = a+b \times m(X)$.

For a linear transformation
𝑎
+
𝑏
𝑋
a+bX, adding
𝑎
a shifts every value of
𝑋
X by the same amount and multiplying by
𝑏
b scales every value. Since the mean is just the average of all values, the mean also shifts by
𝑎
a and scales by
𝑏
b. Therefore, the mean of
𝑎
+
𝑏
𝑋
a+bX is
𝑎
+
𝑏
a+b times the mean of
𝑋
X.

2. Show that $ \text{cov}(X,X) = s^2$.
The covariance of a variable with itself measures how much the variable varies around its own mean. This is exactly what variance captures. Since variance is defined as the average squared deviation from the mean, cov(X,X)\text{cov}(X, X)cov(X,X) is equal to the sample variance s^2.


3. Show that $\text{cov}(X,a+bY) = b \times \text{cov}(X,Y)$

Adding a constant to a variable does not change how it varies with another variable, because the relative differences stay the same. Multiplying by bbb scales those differences by a factor of b. As a result, the covariance between X and a+bY is just b times the covariance between X and Y.


4. Show that $\text{cov}(a+bX,a+bY) = b^2 \text{cov}(X,Y) $. Notice, this also means that $\text{cov}(bX, bX) = b^2 s^2$.

 When both variables are multiplied by bbb, each deviation from the mean is scaled by bbb, so their product is scaled by b^2. This means the covariance of a + bX and a + bY is b^2 times just the covariance of X and Y. In particular, this shows that cov(bX, bX) = b^2s^2

5. Suppose $b>0$ and let the median of $X$ be $\text{med}(X)$. Is it true that the median of $a+bX$ is equal to $a + b \times \text{med}(X)$? Is the IQR of $a + bX$ equal to $a + b \times \text{IQR}(X)$?

When the multiplier is positive, multiplying every value of X keeps the order of the data the same, and adding a constant just shifts all values up or down. Because of this, the median changes in the same way as the data itself. The median of the transformed variable is equal to the constant plus the multiplier times the original median. The interquartile range measures spread, not location, so adding a constant does not change it. However, multiplying by the multiplier does scale the spread. Therefore, the interquartile range of the transformed variable is the multiplier times the original interquartile range, not the constant plus the multiplier times the interquartile range.

6. Show by example that the means of $X^2$ and $\sqrt{X}$ are generally not $(m(X))^2$ and $\sqrt{m(X)}$. So, the results we derived above really depend on the linearity of the transformation $Y = a + bX$, and transformations like $Y = X^2$ or $Y = \sqrt{X}$ will not behave in a similar way.

The mean does not generally commute with nonlinear transformations. For example, if you square each value of a variable and then take the average, that result will usually be different from squaring the original average. The same idea applies to taking square roots: averaging the square roots of the values is not the same as taking the square root of the average. This happens because squaring and square roots change values in uneven ways, especially when the data are spread out. These examples show that the nice properties we proved earlier rely on the transformation being linear, and they do not hold for nonlinear transformations like squaring or taking square roots.

**Q7.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `ames_prices.csv` dataset. The "relevant" variables for this question are:
  - `price` - Sale price value of the house
  - `Bldg.Type` - Building type of the house (single family home, end-of-unit townhome, duplex, interior townhome, two-family conversion)

1. Load the `college_completion.csv` data with Pandas.
2. Make a kernel density plot of price and compute a describe table. Now, make a kernel density plot of price conditional on building type, and use `.groupby()` to make a describe type for each type of building. Which building types are the most expensive, on average? Which have the highest variance in transaction prices?

The kernel density plot of house prices shows a strongly right-skewed distribution, with most homes clustered around $150k–$200k and a long upper tail. When conditioning on building type, end-unit townhomes (TwnhsE) and single-family homes (1Fam) have the highest average prices, while two-family conversions (2fmCon) are the least expensive. Single-family homes also show the largest variance in prices, which is visible in both the wider KDE spread and the higher standard deviation.

3. Make an ECDF plot of price, and compute the sample minimum, .25 quantile, median, .75 quantile, and sample maximum (i.e. a 5-number summary).

The ECDF and five-number summary show a minimum price of $12,789, a first quartile of $129,500, a median of $160,000, a third quartile of $213,500, and a maximum of $755,000. This confirms that most homes are priced well below the extreme upper tail.

4. Make a boxplot of price. Are there outliers? Make a boxplot of price conditional on building type. What patterns do you see?

The overall boxplot of price clearly shows many high-price outliers. When broken down by building type, outliers are most prominent among single-family homes, which also have the widest interquartile range. Townhomes and duplexes tend to have tighter distributions and fewer extreme values.

5. Make a dummy variable indicating that an observation is an outlier.

Using the 1.5×IQR rule, 137 observations were flagged as outliers, while 2,793 were not. This shows that only a small fraction of homes drive the extreme upper tail of the price distribution.

6. Winsorize the price variable, and compute a new kernel density plot and describe table. How do the results change?

After winsorizing prices, the maximum price is capped at $335,000, which substantially reduces the influence of extreme values. The mean and standard deviation decrease, while the median and quartiles remain unchanged. The new kernel density plot is less skewed, indicating that winsorization smooths the distribution without affecting the typical home price.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ames = pd.read_csv("ames_prices.csv")
ames.head()

In [ ]:
ames["price"].plot(kind="kde")
plt.xlabel("House Price")
plt.title("Kernel Density of House Prices")
plt.show()

ames["price"].describe()

In [ ]:
for b in ames["Bldg.Type"].unique():
    ames[ames["Bldg.Type"] == b]["price"].plot(kind="kde", label=b)

plt.legend()
plt.xlabel("House Price")
plt.title("Price Density by Building Type")
plt.show()

ames.groupby("Bldg.Type")["price"].describe()


In [ ]:
ames_sorted = ames["price"].sort_values()
ecdf = ames_sorted.rank(method="average") / len(ames_sorted)

plt.plot(ames_sorted, ecdf)
plt.xlabel("House Price")
plt.ylabel("ECDF")
plt.title("ECDF of House Prices")
plt.show()

ames["price"].quantile([0, 0.25, 0.5, 0.75, 1])

In [ ]:
ames.boxplot(column="price")
plt.title("Boxplot of House Prices")
plt.show()

ames.boxplot(column="price", by="Bldg.Type", rot=45)
plt.title("Price by Building Type")
plt.suptitle("")
plt.show()

In [ ]:
Q1 = ames["price"].quantile(0.25)
Q3 = ames["price"].quantile(0.75)
IQR = Q3 - Q1

ames["outlier"] = (
    (ames["price"] < Q1 - 1.5 * IQR) |
    (ames["price"] > Q3 + 1.5 * IQR)
)

ames["outlier"].value_counts()

In [ ]:
lower = ames["price"].quantile(0.05)
upper = ames["price"].quantile(0.95)

ames["price_wins"] = ames["price"].clip(lower, upper)

ames["price_wins"].plot(kind="kde")
plt.xlabel("Winsorized Price")
plt.title("KDE of Winsorized Prices")
plt.show()

ames["price_wins"].describe()
